In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
import pandas
import datasets

import transformers

from sklearn.metrics import precision_recall_fscore_support, accuracy_score

In [3]:
DATA_FILE: str = "../data/processed/DefaktS_Twitter.binary.csv"
TEST_FRAC: float = 0.10

MODEL_SLUG: str = "Twitter/twhin-bert-base"

OUT_DIR: str = "./fine_tuning_ouput/"

In [4]:
DATA: pandas.DataFrame = (
    pandas.read_csv(DATA_FILE, index_col=[0])
    .rename(columns={"binary_label": "label"})

    # remove urls
    .pipe(lambda _df: _df.assign(
        text=(
            _df["text"]
            # replace urls with special token
            .str.replace(r"https:\/\/t.co\/\S+", "[URL]", regex=True)
        ),
        label=(
            _df["label"].astype(int)
        )
    ))

    # downsample to smallest category
    .pipe(lambda _df: (
        _df
        .groupby("label")
        .sample(n=min(_df["label"].value_counts()))
    ))
)
DATA.head()

,text,label
id,,
428142,"Die Menschen in Belutschistan hören nicht auf,...",0
387854,Im #Iran geht das Regime nicht nur in #Kurdist...,0
407119,US-Jury spricht Elon #Musk im Betrugsprozess u...,0
392035,Führende Fachpolitiker von Grünen und SPD im B...,0
407800,Hyundai Ioniq 6 Electrified Streamliner\nab 29...,0


In [5]:
DATA_TRAIN = DATA.sample(frac=1.0 - TEST_FRAC)
DATA_TEST = DATA.loc[DATA.index.difference(DATA_TRAIN.index)]

DATASET_TRAIN = datasets.Dataset.from_pandas(DATA_TRAIN, split="train")
DATASET_TEST = datasets.Dataset.from_pandas(DATA_TEST, split="test")

len(DATASET_TRAIN), len(DATASET_TEST), DATA_TRAIN.label.nunique()

(14805, 1645, 2)

In [6]:
DATASET_TEST[0:50]

{'text': ['Immer Details, die Abtreibungsfans verschweigen/verstecken:\n[URL]\n"Mädchen war sich der Schwangerschaft nicht bewusst"\n\nOben schreibt man:\n"Doch nun wurde einem Teenager der Schwangerschaftsabbruch trotzdem verwehrt"\n\nNein, ihr wurde Abtreibung nicht verwert,',
  'Wir sind hier in D und nicht in der #UK. Was interessiert hier, wenn in der UK ein Reisesack umfällt. In D wollen wir nicht, das Kinder im Bauch ihrer Mütter getötet werden. Wir verteidigen das #Menschenrecht auf Leben für Alle und bekämpfen es nicht. #Abtreibung ist #MORD',
  'Fristenregelung gut genug: Abtreibung soll in der Schweiz strafbar bleiben [URL]',
  'Es ist wieder mal Zeit für ein teuflisch gutes Thema! - Manches kann man halt nur noch mit einem kräftigen Schluck aus der Sarkasmusflasche ertragen. [URL] #ProChoice #ProLife #AbortionIsHealthcare #Abtreibung',
  '[dieStandard - [URL] Frankreich will Recht auf Abtreibung in der Verfassung verankern [URL]',
  'Ach ja anti Abtreibung natürlich auch le

In [7]:
TOKENIZER = transformers.AutoTokenizer.from_pretrained(MODEL_SLUG)
MODEL = transformers.AutoModelForSequenceClassification.from_pretrained(MODEL_SLUG, num_labels=DATA_TRAIN.label.nunique())

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Twitter/twhin-bert-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
def tokenize_function(sample):
    return TOKENIZER(sample["text"], padding="max_length", truncation=True, max_length=512)

In [9]:
train_tokenized_dataset = DATASET_TRAIN.map(tokenize_function, batched=True)
test_tokenized_dataset = DATASET_TEST.map(tokenize_function, batched=True)

Map:   0%|          | 0/14805 [00:00<?, ? examples/s]

Map:   0%|          | 0/1645 [00:00<?, ? examples/s]

In [10]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro', zero_division=0.0)
    acc = accuracy_score(labels, preds)

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
    }

trainer = transformers.Trainer(
    model=MODEL,
    args=transformers.TrainingArguments(
        num_train_epochs=3,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        output_dir=OUT_DIR,
        overwrite_output_dir=True,
        save_total_limit=1,
        logging_first_step=True,
        logging_steps=50,
        eval_strategy="steps"
    ),
    train_dataset=train_tokenized_dataset,
    eval_dataset=test_tokenized_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,0.594100,0.484997,0.758055,0.754955,0.775115,0.759614
100,0.486000,0.445908,0.794529,0.792783,0.807643,0.795830
